<div class="alert alert-block alert-info">
<b>Deadline:</b> March 20, 2024 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [2]:
skip_training = True  # Set this flag to True before validation and submission

In [3]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the item (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5).

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [7]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [8]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [9]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture=None  # If you want to tune the hyperparameters automatically (e.g. using random
                                    # search), use this argument to specify the hyperparameters that define the
                                    # architecture of your network. After you have tuned the hyperparameters,
                                    # set the default value of this argument to the optimal set of the hyparameters
                                    # so that the best architecture is used in the accuracy tests.
                ):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        # YOUR CODE HERE
        super().__init__()
        self.embedding_dim = 128
        self.dropout = 0.3
        self.nodes = 4
        self.embedding_users = nn.Embedding(n_users+1, self.embedding_dim)
        self.embedding_items = nn.Embedding(n_items+1, self.embedding_dim)
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(self.embedding_dim * 2, self.nodes),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),
            # nn.Linear(self.nodes, self.nodes),
            # nn.ReLU(),
            # nn.Dropout(p=self.dropout),
            nn.Linear(self.nodes, 1)
        )
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        # YOUR CODE HERE
        users_embedded = self.embedding_users(user_ids)
        items_embedded = self.embedding_items(item_ids)
        x = torch.cat([users_embedded, items_embedded], dim = 1)
        x = x.type(torch.FloatTensor)
        x = self.linear_tanh_stack(x)
        return x.squeeze()*5

You can test the shapes of the model outputs using the function below.

In [10]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [11]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [12]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [13]:
# Implement the training loop in this cell
torch.manual_seed(420)
if not skip_training:
    print('start')
    # YOUR CODE HERE
    model = model.to(device)
    n_epochs = 101
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.00001)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=len(trainset), gamma=0.5)
        
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        n = 0
        for users, items, ratings in trainloader:
            users = users.to(device)
            items = items.to(device)
            ratings = ratings.type(torch.FloatTensor)
            ratings = ratings.to(device)
            optimizer.zero_grad()
            outputs = model.forward(users, items)
            loss = nn.MSELoss()(outputs, ratings)
            train_loss += loss
            n += 1
            loss.backward()
            optimizer.step()
            # scheduler.step()
        model.eval()
        train_loss /= n
        with torch.no_grad():
            test_loss = 0
            n = 0
            for users, items, ratings in testloader:
                users = users.to(device)
                items = items.to(device)
                ratings = ratings.type(torch.FloatTensor)
                ratings = ratings.to(device)
                outputs = model.forward(users, items)
                test_loss += nn.MSELoss()(outputs, ratings)
                n += 1
            test_loss /= n
        
            if (epoch) % 5 == 0:
                print(f"Epoch: {epoch}, train_loss: {train_loss:.4f}, test_loss: {test_loss:.4f}")

In [14]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

In [15]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

Model loaded from recsys.pth.


In [16]:
if skip_training or True:
    model = model.to(device)
    model.eval()
    testloader = torch.utils.data.DataLoader(testset, batch_size=64)
    
    with torch.no_grad():
        test_loss = 0
        n = 0
        for users, items, ratings in testloader:
            users = users.to(device)
            items = items.to(device)
            ratings = ratings.type(torch.FloatTensor)
            ratings = ratings.to(device)
            outputs = model.forward(users, items)
            test_loss += nn.MSELoss()(outputs, ratings)
            n += 1
            
    test_loss /= n
    print(f"test_loss: {test_loss:.4f}")

test_loss: 0.8765


The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [17]:
# This cell tests the accuracy of your best model.

In [18]:
# This cell is reserved for grading

In [19]:
# This cell is reserved for grading

In [20]:
# This cell is reserved for grading

In [21]:
# This cell is reserved for grading

In [22]:
# This cell is reserved for grading

In [23]:
# This cell is reserved for grading